In [96]:
# Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from sklearn import metrics 
%matplotlib inline

pd.set_option('display.max_rows', 1000)
#https://kiwidamien.github.io/how-to-do-cross-validation-when-upsampling-data.html

In [97]:
pd.set_option('display.max_rows', 500)

In [107]:
## Update bike collision with bike lane information 

#bikecl = pd.read_csv('./data/bikecollisiondata_with_nearest_street_index2_updated.csv', sep= ",") # bike accidents
# the above file was updated on Google colab to match accidents to closest bikelane 
bikecl = pd.read_csv('./data/bikecl_withlane.csv', sep= ",", na_values= 'None') # bike accidents


In [108]:
bikecl.shape

(5707, 68)

In [109]:
##if there is no match, assume there is no bike facility around 
bikecl['bike_facility'] = bikecl['EXISTING_F']

bikecl.loc[bikecl['cl_bike_match'] == 9.999999e+06, 'bike_facility'] = 'NoFacility'

In [110]:
bikecl.columns

Index(['OBJECTID_x', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS_x', 'ADDRTYPE',
       'INTKEY', 'LOCATION', 'EXCEPTRSNC', 'EXCEPTRSND', 'SEVERITYCO',
       'SEVERITYDE', 'COLLISIONT', 'PERSONCOUN', 'PEDCOUNT', 'PEDCYLCOUN',
       'VEHCOUNT', 'INJURIES', 'SERIOUSINJ', 'FATALITIES', 'INCDATE',
       'INCDTTM', 'JUNCTIONTY', 'SDOT_COLCO', 'SDOT_COLDE', 'INATTENTIO',
       'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTG',
       'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY',
       'CROSSWALKK', 'HITPARKEDC', 'geometry_x', 'near_str_index',
       'min_length', 'weather_group', 'date', 'time', 'hour', 'dayofweek',
       'weekday', 'rushhour', 'weather_group2', 'min_length_cl_to_bike',
       'min_index_cl_to_bike', 'cl_bike_match', 'OBJECTID_y', 'STREET_NAM',
       'LENGTH_MIL', 'STATUS_y', 'COMPKEY', 'CATALYTIC_', 'NETWORK',
       'DATE_COMPL', 'EXISTING_F', 'PLANNED_FA', 'PROJECT_NU', 'DATE_PLANN',
       'DATE_EXPIR', 'SHAPE_Leng', 'geometry_y',

In [111]:
## check accident date and bike facility to update the status of bike faciliy 
bikecl['bikelane_yr_complete'] = pd.to_datetime(bikecl.DATE_COMPL).dt.year
bikecl['accident_year'] = pd.to_datetime(bikecl.INCDTTM).dt.year

In [112]:
bikecl.columns

Index(['OBJECTID_x', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS_x', 'ADDRTYPE',
       'INTKEY', 'LOCATION', 'EXCEPTRSNC', 'EXCEPTRSND', 'SEVERITYCO',
       'SEVERITYDE', 'COLLISIONT', 'PERSONCOUN', 'PEDCOUNT', 'PEDCYLCOUN',
       'VEHCOUNT', 'INJURIES', 'SERIOUSINJ', 'FATALITIES', 'INCDATE',
       'INCDTTM', 'JUNCTIONTY', 'SDOT_COLCO', 'SDOT_COLDE', 'INATTENTIO',
       'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTG',
       'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY',
       'CROSSWALKK', 'HITPARKEDC', 'geometry_x', 'near_str_index',
       'min_length', 'weather_group', 'date', 'time', 'hour', 'dayofweek',
       'weekday', 'rushhour', 'weather_group2', 'min_length_cl_to_bike',
       'min_index_cl_to_bike', 'cl_bike_match', 'OBJECTID_y', 'STREET_NAM',
       'LENGTH_MIL', 'STATUS_y', 'COMPKEY', 'CATALYTIC_', 'NETWORK',
       'DATE_COMPL', 'EXISTING_F', 'PLANNED_FA', 'PROJECT_NU', 'DATE_PLANN',
       'DATE_EXPIR', 'SHAPE_Leng', 'geometry_y',

In [113]:
bikecl.loc[bikecl['accident_year'] < bikecl['bikelane_yr_complete'],['accident_year', 'bikelane_yr_complete', 'EXISTING_F', 'near_str_index' ]]

,accident_year,bikelane_yr_complete,EXISTING_F,near_str_index
8,2012,2013.0,"In Street, Minor Separation",4228.0
26,2016,2017.0,"In Street, Major Separation",17928.0
32,2015,2016.0,"In Street, Major Separation",995.0
41,2015,2016.0,"In Street, Major Separation",15971.0
57,2004,2018.0,"In Street, Major Separation",10130.0
...,...,...,...,...
4669,2016,2017.0,"In Street, Major Separation",3657.0
4721,2017,2018.0,"In Street, Major Separation",16482.0
4762,2016,2017.0,"In Street, Minor Separation",23172.0
4846,2017,2018.0,"In Street, Major Separation",7304.0


In [114]:
bikecl.loc[(bikecl['accident_year'] < bikecl['bikelane_yr_complete']) 
           ,['accident_year', 'bikelane_yr_complete', 'EXISTING_F', 'near_str_index', 'bike_facility' ]]

,accident_year,bikelane_yr_complete,EXISTING_F,near_str_index,bike_facility
8,2012,2013.0,"In Street, Minor Separation",4228.0,"In Street, Minor Separation"
26,2016,2017.0,"In Street, Major Separation",17928.0,"In Street, Major Separation"
32,2015,2016.0,"In Street, Major Separation",995.0,"In Street, Major Separation"
41,2015,2016.0,"In Street, Major Separation",15971.0,"In Street, Major Separation"
57,2004,2018.0,"In Street, Major Separation",10130.0,"In Street, Major Separation"
...,...,...,...,...,...
4669,2016,2017.0,"In Street, Major Separation",3657.0,"In Street, Major Separation"
4721,2017,2018.0,"In Street, Major Separation",16482.0,"In Street, Major Separation"
4762,2016,2017.0,"In Street, Minor Separation",23172.0,"In Street, Minor Separation"
4846,2017,2018.0,"In Street, Major Separation",7304.0,"In Street, Major Separation"


In [115]:
changedstreet_index = bikecl.loc[bikecl['accident_year'] < bikecl['bikelane_yr_complete'],'near_str_index'].reset_index(drop = True)

In [116]:
changedstreet_index = changedstreet_index.astype('int')
changedstreet_index = np.unique(changedstreet_index)

In [117]:
changedstreet_index = np.unique(changedstreet_index)

In [118]:
bikecl.loc[bikecl['accident_year'] < bikecl['bikelane_yr_complete'],'bike_facility'] = 'NoFacility'
## These records do not exist in Current situation, where new bike facilities are built 
## so adding these situations to the street? 

In [76]:
bikecl.groupby('bike_facility').size()

bike_facility
In Street, Major Separation     157
In Street, Minor Separation     820
Multi-use Trail                  49
Neighborhood Greenway            24
NoFacility                     4114
Sharrow                         543
dtype: int64

In [77]:
bikecl.loc[bikecl['bike_facility'].isnull(),'bike_facility'] 

Series([], Name: bike_facility, dtype: object)

In [78]:
bikecl.bike_facility.isnull().sum()

0

In [69]:
## this file has 1) bikecl data matchched with bikelane and 2) index to closest street 
bikecl.to_csv('./data/bikecl_with_bikelane_updated_Oct3.csv', sep= ",")

In [70]:
bikecl.columns

Index(['OBJECTID_x', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS_x', 'ADDRTYPE',
       'INTKEY', 'LOCATION', 'EXCEPTRSNC', 'EXCEPTRSND', 'SEVERITYCO',
       'SEVERITYDE', 'COLLISIONT', 'PERSONCOUN', 'PEDCOUNT', 'PEDCYLCOUN',
       'VEHCOUNT', 'INJURIES', 'SERIOUSINJ', 'FATALITIES', 'INCDATE',
       'INCDTTM', 'JUNCTIONTY', 'SDOT_COLCO', 'SDOT_COLDE', 'INATTENTIO',
       'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTG',
       'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY',
       'CROSSWALKK', 'HITPARKEDC', 'geometry_x', 'near_str_index',
       'min_length', 'weather_group', 'date', 'time', 'hour', 'dayofweek',
       'weekday', 'rushhour', 'weather_group2', 'min_length_cl_to_bike',
       'min_index_cl_to_bike', 'cl_bike_match', 'OBJECTID_y', 'STREET_NAM',
       'LENGTH_MIL', 'STATUS_y', 'COMPKEY', 'CATALYTIC_', 'NETWORK',
       'DATE_COMPL', 'EXISTING_F', 'PLANNED_FA', 'PROJECT_NU', 'DATE_PLANN',
       'DATE_EXPIR', 'SHAPE_Leng', 'geometry_y',

In [119]:
## streets with added sidewalk 

strt2_temp = pd.read_csv('./data/filtedstreets_withsidewalk.csv', sep = "," , na_values= 'None') ## street and sidewalk 

In [120]:
strt2_temp.shape

(23495, 58)

In [121]:
## add CURRENT bikelane information to street databse 
str_bike = pd.read_csv('./data/strt2_withbikelane.csv', sep = ",") #temporary file with link to bikelane 
bikelane = pd.read_csv('./data/Existing_Bike_Facilities.csv', sep= ",")

In [122]:
str_bike.columns

Index(['OBJECTID', 'ARTCLASS', 'COMPKEY', 'UNITIDSORT', 'UNITDESC',
       'STNAME_ORD', 'XSTRLO', 'XSTRHI', 'ARTDESCRIP', 'OWNER', 'STATUS',
       'BLOCKNBR', 'SPEEDLIMIT', 'SEGDIR', 'ONEWAY', 'ONEWAYDIR', 'FLOW',
       'SEGLENGTH', 'SURFACEWID', 'SURFACETYP', 'SURFACET_1', 'INTRLO',
       'DIRLO', 'INTKEYLO', 'INTRHI', 'DIRHI', 'NATIONHWYS', 'STREETTYPE',
       'PVMTCONDIN', 'PVMTCOND_1', 'TRANCLASS', 'TRANDESCRI', 'SLOPE_PCT',
       'PVMTCATEGO', 'PARKBOULEV', 'SHAPE_Leng', 'geometry',
       'closest_bike_index', 'closest_bike_distance'],
      dtype='object')

In [123]:
bikelane['index'] = bikelane.index

In [124]:
# change invalid values to NA
str_bike.loc[str_bike['closest_bike_index'] <= 0 ,'closest_bike_index'] = np.nan
str_bike.groupby('closest_bike_index').size() 

closest_bike_index
1.0       1
2.0       1
3.0       1
4.0       3
5.0       4
         ..
3246.0    2
3248.0    1
3249.0    1
3250.0    1
3252.0    2
Length: 2785, dtype: int64

In [125]:
max(str_bike['closest_bike_index'])
len(bikelane)

3253

In [126]:
## add the index to strt2 data (sidewalk + street)
strt2_temp['closest_bike_index'] = str_bike['closest_bike_index'] 

In [127]:
strt2 = strt2_temp.merge(bikelane, left_on = 'closest_bike_index', right_on = 'index' , how = 'left')

In [128]:
strt2.shape


(23495, 74)

In [129]:
# rename some columns 
strt2 = strt2.rename(columns={"STATUS_y": "STATUS_bike", "STATUS_x": "STATUS_street"})

In [130]:
# clean bike_facility_definition 
strt2.loc[strt2['EXISTING_FACILITY_TYPE']== 'Sharow' ,'EXISTING_FACILITY_TYPE'] = 'Sharrow'
# clean bike_facility_definition 
strt2.loc[strt2['EXISTING_FACILITY_TYPE']== 'sharrow' ,'EXISTING_FACILITY_TYPE'] = 'Sharrow'
strt2.loc[strt2['EXISTING_FACILITY_TYPE'].isnull() ,'EXISTING_FACILITY_TYPE']  = 'NoFacility'
## Clean Speed Limit 
strt2.loc[strt2['SPEEDLIMIT'] == 0 ,'SPEEDLIMIT']  = None

## more cleaning 

strt2.loc[strt2['SURFACEWID'] == 0 ,['SURFACEWID']] = np.nan
strt2.loc[strt2['PVMTCONDIN'] == 0 ,['PVMTCONDIN']]  = np.nan
strt2.loc[strt2['PVMTCOND_1'] == 0 ,['PVMTCOND_1']] = np.nan

In [131]:
## transformation 
strt2['SEGLENGTH_log'] = np.log(strt2['SEGLENGTH'])
strt2['SURFACEWID_log'] = np.log(strt2['SURFACEWID'])

In [132]:
strt2.groupby('EXISTING_FACILITY_TYPE').size()

EXISTING_FACILITY_TYPE
In Street, Major Separation      373
In Street, Minor Separation     1571
Multi-use Trail                  300
Neighborhood Greenway           1009
NoFacility                     18556
Sharrow                         1686
dtype: int64

In [133]:
strt2.shape

(23495, 76)

In [134]:
strt2.columns

Index(['OBJECTID_x', 'ARTCLASS', 'COMPKEY_x', 'UNITIDSORT', 'UNITDESC_x',
       'STNAME_ORD', 'XSTRLO', 'XSTRHI', 'ARTDESCRIP', 'OWNER',
       'STATUS_street', 'BLOCKNBR', 'SPEEDLIMIT', 'SEGDIR', 'ONEWAY',
       'ONEWAYDIR', 'FLOW', 'SEGLENGTH', 'SURFACEWID', 'SURFACETYP',
       'SURFACET_1', 'INTRLO', 'DIRLO', 'INTKEYLO', 'INTRHI', 'DIRHI',
       'NATIONHWYS', 'STREETTYPE', 'PVMTCONDIN', 'PVMTCOND_1', 'TRANCLASS',
       'TRANDESCRI', 'SLOPE_PCT', 'PVMTCATEGO', 'PARKBOULEV', 'SHAPE_Leng',
       'geometry', 'index_x', 'UNITDESC_y', 'STR', 'Side', 'CONDITION',
       'CURBTYPE', 'CURRENT_STATUS', 'FILLERTYPE', 'FILLERWID', 'LENUOM',
       'SW_WIDTH', 'MATL', 'SURFTYPE', 'SWINCOMPLETEYN', 'MULTIPLESURFACEYN',
       'CURBRAMPHIGH', 'CURBRAMPMID', 'CURBRAMPLOW', 'SRTS_SIDEWALK_RANK',
       'PRIMARYCROSSSLOPE', 'MINIMUMVARIABLEWIDTH', 'closest_bike_index',
       'OBJECTID_y', 'STREET_NAME', 'LENGTH_MILES', 'STATUS_bike', 'COMPKEY_y',
       'CATALYTIC_PROJECTS', 'NETWORK', 'DATE_C

In [135]:
strt2.loc[strt2['PVMTCATEGO'] == 'NONE' ,['PVMTCATEGO']  ] = 'NON-ART'
strt2.loc[strt2['PVMTCATEGO'] == 'PATHWAY', ['PVMTCATEGO'] ] = 'NON-ART'
strt2.loc[strt2['PVMTCATEGO'] == 'PVMTTOPARK', ['PVMTCATEGO'] ] = 'NON-ART'

In [136]:
strt2.to_csv('./data/street_bike_sidewalk_oct6.csv', sep = ',', index= False)

In [86]:
bikecl.columns

Index(['OBJECTID_x', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS_x', 'ADDRTYPE',
       'INTKEY', 'LOCATION', 'EXCEPTRSNC', 'EXCEPTRSND', 'SEVERITYCO',
       'SEVERITYDE', 'COLLISIONT', 'PERSONCOUN', 'PEDCOUNT', 'PEDCYLCOUN',
       'VEHCOUNT', 'INJURIES', 'SERIOUSINJ', 'FATALITIES', 'INCDATE',
       'INCDTTM', 'JUNCTIONTY', 'SDOT_COLCO', 'SDOT_COLDE', 'INATTENTIO',
       'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTG',
       'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY',
       'CROSSWALKK', 'HITPARKEDC', 'geometry_x', 'near_str_index',
       'min_length', 'weather_group', 'date', 'time', 'hour', 'dayofweek',
       'weekday', 'rushhour', 'weather_group2', 'min_length_cl_to_bike',
       'min_index_cl_to_bike', 'cl_bike_match', 'OBJECTID_y', 'STREET_NAM',
       'LENGTH_MIL', 'STATUS_y', 'COMPKEY', 'CATALYTIC_', 'NETWORK',
       'DATE_COMPL', 'EXISTING_F', 'PLANNED_FA', 'PROJECT_NU', 'DATE_PLANN',
       'DATE_EXPIR', 'SHAPE_Leng', 'geometry_y',

In [87]:
24111-23495

616

In [88]:
len(bikecl.loc[bikecl['FATALITIES'] > 0,:])/5707*100

0.43805852461888906

In [89]:
# serious or fatalites
(len(bikecl.loc[bikecl['SERIOUSINJ'] > 0,:]) + len(bikecl.loc[bikecl['FATALITIES'] > 0,:]) )/5707*100

7.639740669353426

In [90]:
bikecl['INJURIES'].describe()
len(bikecl.loc[bikecl['INJURIES'] > 0,:])/5707 *100

88.11985281233574

In [91]:
strt2['index'] = strt2.index

In [49]:
# drop 'TRANDESCRI' due to high number of unknown
strt2.groupby('TRANDESCRI').size()

TRANDESCRI
LOCAL TRANSIT ROUTE              184
MAJOR TRANSIT ROUTE             1724
MINOR RESTRICTED TRANSIT RT       13
MINOR TRANSIT ROUTE             3078
NON-ART                           26
NOT DESIGNATED                 18058
PRINCIPAL TRANSIT ROUTE          222
TEMPORARY TRANSIT ROUTE           32
dtype: int64

In [50]:
strt2.groupby('TRANCLASS').size()
strt2.groupby('ARTCLASS').size()
strt2.groupby('PARKBOULEV').size()
strt2.groupby('PVMTCATEGO').size()
strt2.groupby(['PVMTCATEGO', 'ARTCLASS']).size()


PVMTCATEGO  ARTCLASS
ART         0              16
            1            2178
            2            2446
            3            1870
MLTUSETRL   0             147
NON-ART     0           16734
            1               3
            2               3
            3              12
            NON-ART        26
dtype: int64

In [51]:
len(changedstreet_index)

382

In [52]:
tempstr = strt2.iloc[changedstreet_index, :].reset_index(drop = True).copy() 
len(tempstr)

382

In [53]:
tempstr2 = tempstr.loc[tempstr['EXISTING_FACILITY_TYPE'] != 'NoFacility', :].reset_index(drop = True).copy()
tempstr2.head()

,OBJECTID_x,ARTCLASS,COMPKEY_x,UNITIDSORT,UNITDESC_x,STNAME_ORD,XSTRLO,XSTRHI,ARTDESCRIP,OWNER,...,DATE_COMPLETED,EXISTING_FACILITY_TYPE,PLANNED_FACILITY_TYPE,PROJECT_NUMBER,DATE_PLANNED,DATE_EXPIRED,SHAPE_Length,index_y,SEGLENGTH_log,SURFACEWID_log
0,2,2,1009,100150,1ST AVE BETWEEN PIKE ST AND PINE ST,1ST AVE,PIKE ST,PINE ST,Minor Arterial,NaN,...,2014-12-31T00:00:00.000Z,"In Street, Major Separation",NaN,NaN,NaN,NaN,321.874,307,6.05444,4.64439
1,22,2,1296,1500180,10TH AVE E BETWEEN E BLAINE ST AND E HOWE ST,10TH AVE E,E BLAINE ST,E HOWE ST,Minor Arterial,NaN,...,2008-12-31T00:00:00.000Z,"In Street, Minor Separation","In Street, Major Separation",102,NaN,NaN,323.099,974,5.77765,3.73767
2,23,2,1299,1500220,10TH AVE E BETWEEN E BOSTON ST AND E LYNN ST,10TH AVE E,E BOSTON ST,E LYNN ST,Minor Arterial,NaN,...,2008-12-31T00:00:00.000Z,"In Street, Minor Separation","In Street, Major Separation",102,NaN,NaN,417.256,100,6.03309,3.73767
3,40,2,1536,3200140,12TH AVE BETWEEN E MADISON ST AND E PIKE ST,12TH AVE,E MADISON ST,E PIKE ST,Minor Arterial,NaN,...,2009-12-31T00:00:00.000Z,"In Street, Minor Separation",NaN,0,NaN,NaN,443.74,1480,6.09582,3.95124
4,50,3,1659,3400150,12TH AVE S BETWEEN S ATLANTIC ST AND S MASSACH...,12TH AVE S,S ATLANTIC ST,S MASSACHUSETTS ST,Collector Arterial,NaN,...,2009-12-31T00:00:00.000Z,Sharrow,"In Street, Major Separation",122,NaN,NaN,565.95,33,6.33859,3.73767


In [112]:
for i in np.arange(0,len(tempstr2)):
    print([changedstreet_index[i],tempstr2.iloc[i,-10]])
    

[1, 'In Street, Major Separation']
[21, 'In Street, Minor Separation']
[22, 'In Street, Minor Separation']
[28, 'In Street, Minor Separation']
[39, 'Sharrow']
[49, 'Neighborhood Greenway']
[51, 'Sharrow']
[64, 'In Street, Minor Separation']
[88, 'Sharrow']
[92, 'Sharrow']
[126, 'In Street, Major Separation']
[127, 'In Street, Major Separation']
[134, 'Neighborhood Greenway']
[135, 'In Street, Minor Separation']
[162, 'Sharrow']
[180, 'Sharrow']
[212, 'Sharrow']
[262, 'Sharrow']
[397, 'In Street, Minor Separation']
[398, 'In Street, Major Separation']
[481, 'Sharrow']
[482, 'In Street, Minor Separation']
[489, 'Sharrow']
[534, 'Sharrow']
[547, 'Sharrow']
[554, 'Sharrow']
[564, 'Sharrow']
[572, 'In Street, Major Separation']
[627, 'Sharrow']
[632, 'In Street, Minor Separation']
[638, 'Neighborhood Greenway']
[640, 'Sharrow']
[643, 'In Street, Major Separation']
[650, 'Sharrow']
[651, 'In Street, Major Separation']
[670, 'In Street, Minor Separation']
[696, 'In Street, Minor Separation']


In [115]:
for i in np.arange(0,len(tempstr2)):
    tempstr2.iloc[i,-10] = 'NoFacility'
    
strt2_new = pd.concat([strt2, tempstr2],ignore_index = True)

In [119]:
strt2_new.to_csv('./data/street_withlane_Oct5.csv', sep= ",")

In [120]:
len(strt2_new)

23830

## Old

In [ ]:
## increase street size to 2x to include rushhour 
n = 24 * len(strt2)
temp = np.zeros(len(strt2)).tolist()
for i in np.arange(1,24):
    temp += np.zeros(len(strt2)).tolist()

simulate_street = pd.concat([strt2]*24, ignore_index = True)  
simulate_street['hour'] = temp

In [ ]:
n = 24 * len(strt2)
#temp = (np.zeros(int(n*0.5))).tolist() + (np.ones(n- int(n*0.5))).tolist()

In [ ]:
# increase data to 4x to incorporate bike facility 
#simulate_street = pd.concat([strt2]*24, ignore_index = True)  

In [ ]:
#simulate_street['simulate_rushhour'] = temp

In [ ]:
bikecl.columns

In [ ]:
full = pd.merge(simulate_street, bikecl, left_on= ['index','hour'],
                right_on = ['near_str_index','hour'], how ='left', indicator = True)

In [ ]:
bikecl['near_str_index'].describe()

In [ ]:
simulate_street['index']

In [ ]:
len(simulate_street)

In [ ]:
full.loc[full['_merge'] == 'right_only','near_str_index']

In [ ]:
strt2.groupby('EXISTING_FACILITY_TYPE').size()

In [ ]:
full.groupby('_merge').size()

In [ ]:
full['acc'] = [ 1 if i == 'both' else 0 for i in full['_merge']]

In [ ]:
full.groupby('acc').size()

In [ ]:
bikecl['rushhour'].isnull().sum()

In [ ]:
numeric_features = ['SEGLENGTH_log','SURFACEWID_log', 'PVMTCONDIN', 'PVMTCOND_1','SLOPE_PCT' ,
                    'SW_WIDTH' ,'FILLERWID','MINIMUMVARIABLEWIDTH', 'PRIMARYCROSSSLOPE', 'LENGTH_MIL']
#categorical_features = ['ARTCLASS', 'SEGDIR', 'ONEWAY', 'SURFACETYP','NATIONHWYS' ,'PVMTCATEGO', 'PARKBOULEV' ,'TRANDESCRI', 'rushhour']
categorical_features = ['SPEEDLIMIT','SEGDIR', 'STREETTYPE','ONEWAY', 'SURFACETYP','NATIONHWYS' ,'PVMTCATEGO', 'PARKBOULEV' , 'hour',
'CONDITION','SURFTYPE', 'SWINCOMPLETEYN', 'MATL' , 'FILLERTYPE', 'CURBTYPE', 'EXISTING_FACILITY_TYPE']
X = full[numeric_features + categorical_features ].copy()
y = full['acc']

In [ ]:
#print(X.dtypes[full.dtypes == object])

In [ ]:
full.groupby(['acc']).size()

In [ ]:
len(strt2)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,test_size=0.1)

In [ ]:
y_train = y_train.astype('int')

In [ ]:
# initial parameters
RANDOM_STATE = 42
#RATIO = 0.1
#K_FOLD = 5
from sklearn.model_selection import StratifiedKFold
K_FOLD = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline


## pre-processing 

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
# imbalanced learn 
rus = RandomUnderSampler(random_state=RANDOM_STATE) #ratio = RATIO

In [ ]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(solver='liblinear')

In [ ]:
## Logsitic Regression
from imblearn.pipeline import make_pipeline

logReg = make_pipeline(preprocessor,rus, log)
#logReg = Pipeline(steps=[('preprocessor', preprocessor),('undersampler', rus),('classifier', log)])

In [ ]:
## cross validate 
from sklearn.model_selection import cross_validate

scores = cross_validate(logReg, X_train, y_train, scoring=['roc_auc','recall','accuracy'], cv = K_FOLD, return_train_score= True, return_estimator=True)

In [ ]:
scores

In [ ]:
## Random Oversampler 

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=RANDOM_STATE) #ratio = RATIO

## Logsitic Regression
from imblearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression
log = LogisticRegression(solver='liblinear')

logReg_ros = make_pipeline(preprocessor,ros, log)

## cross validate 
from sklearn.model_selection import cross_validate

scores = cross_validate(logReg_ros, X_train, y_train, scoring=['roc_auc','recall','accuracy'], cv = K_FOLD, return_train_score= True, return_estimator=True)

In [ ]:
scores

In [ ]:
## SMOTE 

from imblearn.over_sampling import SMOTE, ADASYN,KMeansSMOTE

sm = SMOTE(random_state=42)
#sm = KMeansSMOTE(sampling_strategy= 'minority', random_state=42)
## Logsitic Regression
from imblearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression
log = LogisticRegression(solver='liblinear')

logReg_sm = make_pipeline(preprocessor,sm, log)

## cross validate 
from sklearn.model_selection import cross_validate

scores = cross_validate(logReg_sm, X_train, y_train, scoring=['roc_auc','recall','accuracy'], cv = K_FOLD, return_train_score= True, return_estimator=True)

In [ ]:
scores

In [ ]:
## Random Forest 
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=RANDOM_STATE)
rf = make_pipeline(preprocessor,rus, clf)
## cross validate 
from sklearn.model_selection import cross_validate

scores = cross_validate(rf, X_train, y_train, scoring=['roc_auc','recall','accuracy','precision', 'f1'], cv = K_FOLD, return_train_score = True, return_estimator=True)

In [ ]:
scores 

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [4, 6, 10, 12],
    'random_state': [13]
}
new_params = {'randomforestclassifier__' + key: params[key] for key in params}
grid_imba = GridSearchCV(rf, param_grid=new_params, cv=K_FOLD, scoring='recall', return_train_score=True)
grid_imba.fit(X_train, y_train);

In [ ]:
grid_imba.best_params_

In [ ]:
grid_imba.best_score_

In [ ]:
f_imp = grid_imba.best_estimator_['randomforestclassifier'].feature_importances_

In [ ]:
cat_name = grid_imba.best_estimator_.named_steps['columntransformer'].transformers_[1][1].named_steps['onehot'].get_feature_names()


In [ ]:
featurename = np.array(np.concatenate(( np.array(numeric_features), cat_name,)))

In [ ]:
featurename 

In [ ]:
categorical_features

In [ ]:
feat_imp = pd.DataFrame({
    'Feature': featurename [np.argsort(f_imp)],
    'Importance': np.sort(f_imp)
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10,10))
plt.barh(y = feat_imp['Feature'], width=feat_imp['Importance'])
plt.gca().invert_yaxis()
plt.xlabel('Feature importance')
plt.title('Random forest feature importance')
#plt.savefig("fig/rf_feat_importance.png", bbox_inches='tight')

In [ ]:
featurename [np.argsort(f_imp)]

In [ ]:
np.sort(f_imp)

In [ ]:
logReg.fit(X_train, y_train)

In [ ]:
prediction = logReg.predict(X_test)
len(prediction)


In [ ]:
metrics.recall_score(y_test, prediction)

In [ ]:
metrics.precision_score(y_test, prediction)

In [ ]:
metrics.roc_auc_score(y_test, prediction)

In [ ]:
metrics.f1_score(y_test, prediction)

In [ ]:
metrics.confusion_matrix(y_test, prediction)

In [ ]:

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax




In [ ]:
np.set_printoptions(precision=2)



In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, prediction, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, prediction, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
log.coef_.T

In [ ]:
X_train.columns

In [ ]:
logReg.predict_proba(X_train)
#https://www.guru99.com/scikit-learn-tutorial.html

In [ ]:
cat_name = logReg.named_steps['columntransformer'].transformers_[1][1].named_steps['onehot'].get_feature_names()
featurename = np.array(np.concatenate(( np.array(numeric_features), cat_name,)))

In [ ]:
featurename = np.array(np.concatenate(( np.array(numeric_features), cat_name,)))

In [ ]:
featurename.size

In [ ]:
type(logReg.named_steps['columntransformer'].transformers_[1][1].named_steps['onehot'].get_feature_names())

In [ ]:
plt.figure(figsize=(6,6))
plt.barh(range(len(log.coef_[0])), width=np.sort(log.coef_[0]))
plt.yticks(ticks=range(len(log.coef_[0])), labels=featurename[np.argsort(log.coef_[0])])
plt.title('Logistic regression coefficients')
print(featurename[np.argsort(log.coef_[0])])
print(np.sort(log.coef_[0]))
#plt.savefig("fig/lr_coeff.png", bbox_inches='tight')

In [ ]:
featurename[np.argsort(log.coef_[0])]

In [ ]:
np.argsort(log.coef_[0])

In [ ]:
np.argsort(log.coef_[0])

In [ ]:
range(len(log.coef_[0]))

In [ ]:
log.coef_[0]